# Predict with pre-trained Mask RCNN models

Mask RCNN networks are extensions to Faster RCNN networks. :py:class:gluoncv.model_zoo.MaskRCNN is inherited from :py:class:gluoncv.model_zoo.FasterRCNN. It is highly recommended to read :doc:../examples_detection/demo_faster_rcnn first.

First let's import some necessary libraries:

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils

# Load a pretrained model

Let's get an Mask RCNN model trained on COCO dataset with ResNet-50 backbone. By specifying pretrained=True, it will automatically download the model from the model zoo if necessary. For more pretrained models, please refer to :doc:../../model_zoo/index.

The returned model is a HybridBlock :py:class:gluoncv.model_zoo.MaskRCNN with a default context of cpu(0).

In [2]:
net = model_zoo.get_model('mask_rcnn_resnet50_v1b_coco', pretrained=True)

100%|██████████| 131497/131497 [00:04<00:00, 28393.09KB/s]


# Automated Process
----------------
This part will automate the process of loading images in input folder, doing instance segmentations, saving a summary result csv, and saving output images with masks)


## !!! Please read before running this cell
Make sure that the path_in variable is set to the folder containing your images.

Make sure a folder called 'MaskRCNN' is created in the same folder as the Jupyter notebook file.

In [3]:
# 1. Load all downloaded images
from os import listdir
from os.path import isfile, join
import pandas as pd

# Edit this to fit your environment
path_in='Images/'
path_out='MaskRCNN/'

all_files = [f for f in listdir(path_in)]
# Get only jpeg files
jpg_files = list(filter(lambda x: x[-5:] == ('.jpeg') or x[-4:] == ('.jpg'), all_files))
jpg_files.sort()
#print(jpg_files) 
print("Number of Image Files in the folder:",len(jpg_files))


Number of Image Files in the folder: 4


## ! Please read before running this cell !

### Change the starrNum (Start Number) and endNum (End number) to the index of the images that you intended to process

Check the return value from the cell above for the total number of images indexed.

In [4]:
# 2. Automate the process
# Please change the following parameters
startNum = 0 # Starting index
endNum = 4 # Ending index

# 2.1 Define the features if you want
keywords=['person','bicycle','car','motorcycle','bus','truck','boat','traffic light','dog','dining table']

# 2.2 Load images in the folder, and build an empty dataframe to save the results
import pandas as pd
df_mcnn=pd.DataFrame()

ID=0 # The index we use to save MaskCNN results

for filename in jpg_files[startNum:endNum]:
    # 1. Loading images
    # 1.1 Define Image path
    im_fname = path_in+filename
#     print(im_fname, filename)

    # 1.2 Load image data and feed into x (transformed NDArray with shape) and orig_img (image itself)
    x,orig_img = data.transforms.presets.rcnn.load_test(im_fname)
    
    # 1.3 Display the image
    #plt.imshow(orig_img)
    #plt.show()
    
    # 1.4 Get the image name as FID
    FID=(filename.split('.')[0])

    # 1.5 Start a new row in the dataframe
    df_mcnn.loc[ID,'FID']=FID
       
    # 2. MaskRCNN analyze a single image
    
    # Optional Timer
#     from datetime import datetime
#     startTime = datetime.now() # initiate timer
    print('Processing: %d/%d - %s'%((ID+1),(endNum-startNum),filename))

    ### 2.1 Get MaskRcnn Results
    ids, scores, bboxes, masks = [xx[0].asnumpy() for xx in net(x)]

    ### 2.2 Paint Segmentation Mask on images Directly
    width, height = orig_img.shape[1], orig_img.shape[0]
    masks, _ = utils.viz.expand_mask(masks, bboxes, (width, height), scores)
    orig_img = utils.viz.plot_mask(orig_img, masks)

    ### 2.3 Identical to Faster RCNN object detection
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax = utils.viz.plot_bbox(orig_img, bboxes, scores, ids,
                             class_names=net.classes, ax=ax)

    # Stop optional timer
#     print("runtime:",datetime.now()-startTime) # Print stop timer

    ### 2.4 Save the MaskRCNN image
    fig.savefig("MaskRCNN/"+FID+".png",bbox_inches='tight')
    plt.close()
    #plt.show()
    
    ### 2.5 Get the list of validated results with probability likelihood >0.5
    MCNN_List=[]
    for i in range(len(ids)):
        classID,classscore=(int(ids[i]),scores[i])
        if classscore>0.6: 
            className=net.classes[classID]
            MCNN_List.append(className)
                
    
    ### 2.6 Build a dataframe to save the result
    import collections, numpy    
    unique, counts = numpy.unique(MCNN_List, return_counts=True)
    for i in range(len(unique)):
        cls_name=unique[i]
        cnt=int(counts[i])
        df_mcnn.loc[ID,cls_name]=cnt
        
    print('Finish %s, start next photo'%(filename))    
    ID+=1
    
# 2.8 Save the MaskCNN results to csv file
df_mcnn.to_csv('mcnn_results_%d-%d.csv'%(startNum,endNum-1), index=False, encoding='utf-8')
print('---------COMPLETED !!!----------')
print('Successfully saved mcnn_results_%d-%d.csv'%(startNum,endNum-1))
df_mcnn.head()

Processing: 1/4 - 15730.jpg
Finish 15730.jpg, start next photo
Processing: 2/4 - 16565.jpg
Finish 16565.jpg, start next photo
Processing: 3/4 - 5219.jpg
Finish 5219.jpg, start next photo
Processing: 4/4 - 6829.jpg
Finish 6829.jpg, start next photo
---------COMPLETED !!!----------
Successfully saved mcnn_results_0-3.csv


,FID,bench,bird,car,bicycle,bus,person,handbag,fire hydrant,truck
0,15730,4.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,16565,NaN,NaN,3.0,1.0,1.0,7.0,NaN,NaN,NaN
2,5219,NaN,NaN,13.0,NaN,NaN,10.0,1.0,NaN,NaN
3,6829,NaN,NaN,8.0,NaN,NaN,NaN,NaN,1.0,3.0


# End of this week's assignment